In [6]:
import neptune

NEPTUNE_API_TOKEN="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxMGM5ZDhiMy1kOTlhLTRlMTAtOGFlYy1hOTQzMDE1YjZlNjcifQ=="


In [3]:
model = neptune.init_model(
    # name="Pretrained Model",
    # key="MOD", 
    with_id='SUR-MOD',
    project="revivemed/Survival-RCC", 
    api_token=NEPTUNE_API_TOKEN, # your credentials,

)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/Survival-RCC/m/SUR-MOD


In [25]:
run= neptune.init_run(
        project='revivemed/RCC',
        api_token= NEPTUNE_API_TOKEN,
        with_id='RCC-2925',
        mode='read-only',
)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/revivemed/RCC/e/RCC-2925


In [26]:
run_struc = run.get_structure()

In [27]:
def check_if_path_in_struc(run_struc,target_path):
    target_keys = target_path.split('/')
    for key in target_keys:
        if key in run_struc:
            run_struc = run_struc[key]
        else:
            return False
    return True

In [28]:
from collections import defaultdict
import numpy as np
import pandas as pd

In [29]:
num_trials_key = 'original_kwargs/optimized_study_info/number of total trials'
chosen_trials_key = 'original_kwargs/optimized_study_info/best trial'
res_dict = defaultdict(dict)
run_struc = run.get_structure()
for task_key in run_struc.keys():
    if 'Optimized_' in task_key:
        
        if check_if_path_in_struc(run_struc[task_key],num_trials_key):
            num_trials = run[task_key][num_trials_key].fetch()
            chosen_trial = run[task_key][chosen_trials_key].fetch()
            res_dict[task_key]['num_trials'] = num_trials
            res_dict[task_key]['chosen_trial'] = chosen_trial

        if 'avg' in run_struc[task_key].keys():
            eval_res_dict = run[task_key]['avg'].fetch()
            res_dict[task_key].update(eval_res_dict)


pd.DataFrame(res_dict).T.to_csv('~/Desktop/neptune_results 3.csv')

In [22]:
pd.DataFrame(res_dict).T.to_csv('~/Desktop/neptune_results.csv')

In [3]:
val_array = run['Optimized_EVER-OS_finetune/eval/val2/reconstruction_loss'].fetch_values()

In [5]:
val_array['value'].mean()

1.087329661846161

In [3]:
def count_fields(d):
    return sum([count_fields(v) if isinstance(v, dict)
                     else 1 for v in d.values()])

In [4]:
count_fields(run.get_structure())

5251

In [5]:
run.stop()

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-2925/metadata


In [ ]:
model['original_kwargs'] = run['pretrain/original_kwargs'].fetch()

In [6]:
run_struc = run.get_structure()

for set_name in ['train','val','test']:
    for key in run_struc['pretrain']['eval'][set_name].keys():
        # print(key)
        model['evaluation/'+set_name+'/'+key] = run['pretrain/eval/'+set_name+'/'+key].fetch_last()

In [ ]:
input_dir = '/Users/jonaheaton/Desktop/model info/'

In [ ]:
model["model/encoder_state"].upload(input_dir+'encoder_state_dict.pth')
model["model/heads/Binary_isFemale_state"].upload(input_dir+'Binary_isFemale_state.pt')
model["model/heads/Binary_isPediatric_state"].upload(input_dir+'Binary_isPediatric_state.pt')
model["model/heads/MultiClass_Cohort Label_state"].upload(input_dir+'MultiClass_Cohort Label_state.pt')


In [ ]:
model["model/heads/Regression_Age_state"].upload(input_dir+'Regression_Age_state.pt')
model["model/adversary/MultiClass_Adv StudyID_state"].upload(input_dir+'MultiClass_Adv StudyID_state.pt')

In [ ]:
model['evaluation/test/Z_embed'].upload(input_dir+'Z_embed_test.csv')
model['evaluation/train/Z_embed'].upload(input_dir+'Z_embed_train.csv')
model['evaluation/val/Z_embed'].upload(input_dir+'Z_embed_val.csv')


In [7]:
model.stop()
run.stop()

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/Survival-RCC/m/SUR-MOD/metadata
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/revivemed/RCC/e/RCC-2925/metadata
